In [1]:
import nibabel as nib #pip install nibabel
import numpy as np
import matplotlib.pyplot as plt
#https://openneuro.org/datasets/ds002739/versions/1.0.0

### Load the data


In [2]:
#get current path
import os
current_ = os.getcwd()

In [7]:
img = nib.load(current_ +'/sub-01_task-main_run-01_bold.nii')
data = np.asanyarray(img.dataobj)
print(data.shape)

(70, 70, 32, 771)


### Preprocess the data

In [8]:
vol_shape = data.shape[:-1]
n_vols = data.shape[-1]
print(vol_shape, n_vols)

(70, 70, 32) 771


In [ ]:
#need the time axis -> the events data?

In [ ]:
N = np.prod(vol_shape)

#reshape in function of the time axis and take the transpose

In [ ]:
#to get the PCA need the row means in a times_axis_shape x N
# use np.outer(row_means, np.ones(N))

#substract the mean from the time axis
X = first_2 - row_means

In [ ]:
#plot X[0] vs X[1] 

### PCA 

In [ ]:
# compute the covariance matrix
covariance_matrix = X.dot(X.T)

# Get the U, S, VT matrices
U, S, VT = np.linalg.svd(covariance_matrix)

In [ ]:
# plot the signal in the first row against 
#signal in the second row

plt.plot(X[0], X[1], 'o')
scaled_u = U[0, : ] * -4000
plt.plot([0, scaled_u[0]], [0, scaled_u[1]], 'r')

In [ ]:
# Scalar projection
C = U.T.dot(first_two)

C_vols = C.T.reshape(vol_shape + (2,))

# break 4D array in 3D
vol0 = C_vols[..., 0]
vol1 = C_vols[..., 1]

In [ ]:
plt.imshow(vol0[:, :, 10], cmap='gray')

In [ ]:
plt.imshow(vol1[:, :, 10], cmap='gray')

### Spatial & Temporal ICA

### Group ICA

In [ ]:
#!! need contrast map and T1-weighted image to plot